In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torchsummary import summary
from torch.cuda.amp import autocast, GradScaler

import numpy as np
import gzip
import pickle
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gc
import time
import random
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix



import sys
sys.path.append('..')
# from slp_package.slp_functions import create_merged_game_data_df
from slp_package.input_dataset import InputDataSet
import slp_package.pytorch_functions as slp_pytorch_functions

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using CUDA
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
source_data = ['ranked','public','mango']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
    'num_players': [2],
    'conclusive': [True],
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    # 'character_name': ['FALCO'],
    # 'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'MARIO', 'PEACH', 'ROY', 'LUIGI', 'YOUNG_LINK', 'DONKEY_KONG', 'PICHU', 'KIRBY'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'PEACH', 'LUIGI', 'DONKEY_KONG'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    # 'character_name': ['MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0']
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['player'], # Can be 'general', 'player
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}

In [ ]:
dataset = InputDataSet(source_data, general_features, player_features, opposing_player_features, label_info)

print(dataset.dataset['labels'].value_counts())

In [ ]:
dataset.dataset.head()

In [ ]:
labels_order =  dataset.number_of_segments_per_game(60,5000)
print(labels_order)
labels_order = labels_order['Label'].values


In [ ]:
train_df, test_df  = dataset.train_test_split_dataframes(test_ratio = .20, val = False)

In [ ]:
# train_df, test_df = dataset.all_segments_train_test_split_dataframes(60, proportion_of_segments=.01, test_ratio = .20, val = False)

In [ ]:
print(train_df.shape)
print(test_df.shape)
print(test_df.shape[0] / (train_df.shape[0] + test_df.shape[0]))

In [ ]:
train_df.head()

In [ ]:
class TrainingDataset(Dataset):
    """
    Custom dataset for loading game segments from compressed numpy files.
    """
    def __init__(self, df, transform=None):
        self.file_paths = df['player_inputs_np_sub_path'].to_numpy()
        self.encoded_labels = df['encoded_labels'].to_numpy()
        self.segment_start_index = df['segment_start_index'].to_numpy()
        # self.segment_index = df['segment_index'].to_numpy()
        self.segment_length = df['segment_length'].to_numpy()
        self.transform = transform

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.file_paths)
    

    def __getitem__(self, idx):
        """Loads and returns a sample from the dataset at the specified index."""
        with gzip.open('/workspace/melee_project_data/input_np/' + self.file_paths[idx].replace('\\','/'), 'rb') as f:
            segment = np.load(f)

        
            
        
        # Start and end of the segment
        segment_start = self.segment_start_index[idx]
        segment_end = self.segment_start_index[idx] + self.segment_length[idx]
        
        segment = segment[:,segment_start:segment_end]
        
        if self.transform:
            transformed = np.zeros((13,60))
            transformed[0,:] = (np.abs(segment[0]) - .2875) * 1.40350877193 * (segment[0] > 0)
            transformed[1,:] = (np.abs(segment[0]) - .2875) * 1.40350877193 * (segment[0] < 0)
            transformed[2,:] = (np.abs(segment[1]) - .2875) * 1.40350877193 * (segment[1] > 0)
            transformed[3,:] = (np.abs(segment[1]) - .2875) * 1.40350877193 * (segment[1] < 0)
            transformed[4,:] = (np.abs(segment[2]) - .2875) * 1.40350877193 * (segment[2] > 0)
            transformed[5,:] = (np.abs(segment[2]) - .2875) * 1.40350877193 * (segment[2] < 0)
            transformed[6,:] = (np.abs(segment[3]) - .2875) * 1.40350877193 * (segment[3] > 0)
            transformed[7,:] = (np.abs(segment[3]) - .2875) * 1.40350877193 * (segment[3] < 0)
            transformed[8:,:] = segment[4:]
            segment = transformed
            
        
        # Convert to PyTorch tensors
        segment_tensor = torch.from_numpy(segment).float()
        # label_tensor = torch.tensor(self.encoded_labels[idx], dtype=torch.long)
        return segment_tensor#, label_tensor
    
def prepare_data_loaders(train_df, test_df, batch_size, num_workers):
    # Initialize datasets
    train_dataset = TrainingDataset(train_df,True)
    # val_dataset = TrainingDataset(file_paths_val, labels_val)
    test_dataset = TrainingDataset(test_df,True)

    # Initialize data loaders
    loaders = {
        'train': DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=True,persistent_workers=True),
        'test': DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, pin_memory=True,persistent_workers=True),
        # 'val': DataLoader(val_dataset, batch_size=2**9, num_workers=num_workers, shuffle=False, pin_memory=True,persistent_workers=True)
    }
    return loaders



# ''' Get a batch of data to see the size if we want that information. ''' 
# data_loader_iterator = iter(loaders['train'])
# first_batch = next(data_loader_iterator)
# print(first_batch.shape)



In [ ]:
def train_model(model, criterion, optimizer, loaders, device, num_epochs=1):
    scaler = GradScaler()  # Initialize the gradient scaler

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_loader_tqdm = tqdm(loaders['train'], desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')
        total = 0
        i = 3
        for batch_number, target_cpu in enumerate(train_loader_tqdm):
            target_gpu = target_cpu.to(device)
            
            # Resets the optimizer
            optimizer.zero_grad()
            
            # Runs the forward pass with autocasting.
            with autocast():
                output_gpu = model(target_gpu)
                loss = criterion(output_gpu , target_gpu )
                # print(loss)
            
            # Scales loss and calls backward() to create scaled gradients
            scaler.scale(loss).backward()
            
            # Clip gradients to avoid explosion
            scaler.unscale_(optimizer)  # unscale the gradients of optimizer's assigned params
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            # Before calling step(), check for inf or NaN values in the gradients
            if any(torch.isinf(p.grad).any() for p in model.parameters() if p.grad is not None):
                print("Warning: inf values in gradients!")
            elif any( torch.isnan(p.grad).any() for p in model.parameters() if p.grad is not None):
                print("Warning: NaN values in gradients!")
                
            # scaler.step() first unscales the gradients of the optimizer's assigned params.
            # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
            # otherwise, optimizer.step() is skipped.
            scaler.step(optimizer)
            
            # Updates the scale for next iteration.
            scaler.update()

            # Update progress
            train_loss += loss.item()
            total += target_gpu.size(0)
            train_loader_tqdm.set_postfix(loss=f'{train_loss / (total):.4f}')
            # i += 1
            # if i > 10:
            #     break


def evaluate_model(model, criterion, loaders, loader, device):
    model.eval()
    eval_loss = 0
    total = 0
    with torch.no_grad():
        eval_loader_tqdm = tqdm(loaders[loader], unit = 'batch')
        
        for batch_number, target_cpu in enumerate(eval_loader_tqdm):
            target_gpu = target_cpu.to(device)
            output_gpu = model(target_gpu)
            
            eval_loss += criterion(output_gpu, target_gpu).item()
            total += target_gpu.size(0)
            eval_loader_tqdm.set_postfix(loss=f'{eval_loss / (total):.4f}') 
            
    print(f'Evaluated Loss: {eval_loss / total:.6f}')
    

In [ ]:
from ResNet_Autoencoder_Model import ResNet_Autoencoder

# Build model
model = ResNet_Autoencoder(13).to('cuda')

# With the size of an input we can get a model summary.
# summary(model, input_size=(9, 60))

# Check that the output shape and target shape match
# training_example = torch.rand(9, 2 ** 12).to('cuda')
# print('Target shape:', training_example.shape)
# model.eval()
# output = model(training_example)
# print('Output shape:', output.shape)

## Optionally compile the model
# import torch_tensorrt
# model = torch.compile(model, mode = 'default')
# model = torch.compile(model,mode = 'max-autotune')
# model = torch.compile(model, backend="torch_tensorrt")
# model = torch.compile(model, backend="torch_tensorrt",mode = 'max-autotune')


In [ ]:
# import torch._dynamo
# torch._dynamo.config.suppress_errors = True

torch.autograd.set_detect_anomaly(False)
# Pepare data loaders
batch_size =  32 * 16 * 5 
batch_size =  32 
num_workers = 22
loaders = prepare_data_loaders(train_df, test_df, batch_size, num_workers)

criterion = nn.MSELoss(reduction = 'sum')

class LInfinityLoss(nn.Module):
    def __init__(self):
        super(LInfinityLoss, self).__init__()

    def forward(self, predictions, targets):
        # Compute the infinity norm of the difference
        return torch.max(torch.abs(predictions - targets))

class LPNormLoss(nn.Module):
    def __init__(self, p=2):
        super(LPNormLoss, self).__init__()
        self.p = p

    def forward(self, predictions, targets):
        # Compute the lp norm of the difference
        return torch.norm(predictions - targets, p=self.p)

# Example usage:
# criterion = LPNormLoss(p=6) 
# criterion = LInfinityLoss()
optimizer = Adam(model.parameters(), lr=0.001)
num_epochs = 1

# # # This seems to sometimes help
# gc.collect()
# torch.cuda.empty_cache()
# Train the model
# start_time = time.time()
train_model(model, criterion, optimizer, loaders, 'cuda', num_epochs)
# print(f'Batch Size: {batch_size}, Training time: {time.time() - start_time:.2f}')

# Again, this sometimes seems to help
# gc.collect()
# torch.cuda.empty_cache()

# Evaluate the trained model
# evaluate_model(model, criterion, loaders, 'test', 'cuda')

In [ ]:
def predict(model, loaders, loader, device):
    model.eval()
    predictions = []
    targets = []
    
    with torch.no_grad():
        eval_loader_tqdm = tqdm(loaders[loader], unit='batch')
        
        for _, target_cpu in enumerate(eval_loader_tqdm):
            target_gpu = target_cpu.to(device)
            output_gpu = model(target_gpu)
            
            predictions.append(output_gpu.cpu().numpy())
            targets.append(target_cpu.numpy())
    
    predictions = np.concatenate(predictions, axis=0)
    targets = np.concatenate(targets, axis=0)
    
    return predictions, targets

In [ ]:
pred, target = predict(model, loaders, 'test','cuda')

In [ ]:
evaluate_model(model, criterion, loaders, 'test', 'cuda')

In [ ]:
print(type(pred))
print(pred.shape)
print(type(target))
print(target.shape)

In [ ]:
def stick_histograms(predictions, targets, jstick_n_bins, c_stick_n_bins):
    
    
    # # get the max and min of JSTICK_X, JSTICK_Y, CSTICK_X, and CSTICK_Y
    jstick_x_max, jstick_x_min = np.max(predictions[:, 0, :]), np.min(predictions[:, 0, :])
    jstick_y_max, jstick_y_min = np.max(predictions[:, 1, :]), np.min(predictions[:, 1, :])
    cstick_x_max, cstick_x_min = np.max(predictions[:, 2, :]), np.min(predictions[:, 2, :])
    cstick_y_max, cstick_y_min = np.max(predictions[:, 3, :]), np.min(predictions[:, 3, :])
    
    print(f'Predictions JSTICK_X max: {jstick_x_max}, min: {jstick_x_min}')
    print(f'Predictions JSTICK_Y max: {jstick_y_max}, min: {jstick_y_min}')
    print(f'Predictions CSTICK_X max: {cstick_x_max}, min: {cstick_x_min}')
    print(f'Predictions CSTICK_Y max: {cstick_y_max}, min: {cstick_y_min}')
    
    predictions = np.clip(predictions,-1,1)
    # Histograms
    pred_hist_jstick_x, edges_jstick_x = np.histogram(predictions[:, 0, :], bins=jstick_n_bins, range=(-1, 1))
    target_hist_jstick_x, _ = np.histogram(targets[:, 0, :], bins=jstick_n_bins, range=(-1, 1))
    
    pred_hist_jstick_y, edges_jstick_y = np.histogram(predictions[:, 1, :], bins=jstick_n_bins, range=(-1, 1))
    target_hist_jstick_y, _ = np.histogram(targets[:, 1, :], bins=jstick_n_bins, range=(-1, 1))
    
    pred_hist_cstick_x, edges_cstick_x = np.histogram(predictions[:, 2, :], bins=c_stick_n_bins, range=(-1, 1))
    target_hist_cstick_x, _ = np.histogram(targets[:, 2, :], bins=c_stick_n_bins, range=(-1, 1))
    
    pred_hist_cstick_y, edges_cstick_y = np.histogram(predictions[:, 3, :], bins=c_stick_n_bins, range=(-1, 1))
    target_hist_cstick_y, _ = np.histogram(targets[:, 3, :], bins=c_stick_n_bins, range=(-1, 1))
    
    # Plotting histograms
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    
    # JSTICK_X histogram
    axs[0, 0].bar(edges_jstick_x[:-1], pred_hist_jstick_x, width=np.diff(edges_jstick_x), edgecolor="black", align="edge", label='Predictions')
    axs[0, 0].bar(edges_jstick_x[:-1], target_hist_jstick_x, width=np.diff(edges_jstick_x), edgecolor="red", align="edge", label='Targets', alpha=0.5)
    axs[0, 0].legend()
    axs[0, 0].set_title('Histogram of JSTICK_X')
    axs[0, 0].set_xlabel('JSTICK_X values')
    axs[0, 0].set_ylabel('Frequency')
    axs[0, 0].set_yscale('log')
    
    # JSTICK_Y histogram
    axs[0, 1].bar(edges_jstick_y[:-1], pred_hist_jstick_y, width=np.diff(edges_jstick_y), edgecolor="black", align="edge", label='Predictions')
    axs[0, 1].bar(edges_jstick_y[:-1], target_hist_jstick_y, width=np.diff(edges_jstick_y), edgecolor="red", align="edge", label='Targets', alpha=0.5)
    axs[0, 1].legend()
    axs[0, 1].set_title('Histogram of JSTICK_Y')
    axs[0, 1].set_xlabel('JSTICK_Y values')
    axs[0, 1].set_ylabel('Frequency')
    axs[0, 1].set_yscale('log')
    
    # CSTICK_X histogram
    axs[1, 0].bar(edges_cstick_x[:-1], pred_hist_cstick_x, width=np.diff(edges_cstick_x), edgecolor="black", align="edge", label='Predictions')
    axs[1, 0].bar(edges_cstick_x[:-1], target_hist_cstick_x, width=np.diff(edges_cstick_x), edgecolor="red", align="edge", label='Targets', alpha=0.5)
    axs[1, 0].legend()
    axs[1, 0].set_title('Histogram of CSTICK_X')
    axs[1, 0].set_xlabel('CSTICK_X values')
    axs[1, 0].set_ylabel('Frequency')
    axs[1, 0].set_yscale('log')
    
    # CSTICK_Y histogram
    axs[1, 1].bar(edges_cstick_y[:-1], pred_hist_cstick_y, width=np.diff(edges_cstick_y), edgecolor="black", align="edge", label='Predictions')
    axs[1, 1].bar(edges_cstick_y[:-1], target_hist_cstick_y, width=np.diff(edges_cstick_y), edgecolor="red", align="edge", label='Targets', alpha=0.5)
    axs[1, 1].legend()
    axs[1, 1].set_title('Histogram of CSTICK_Y')
    axs[1, 1].set_xlabel('CSTICK_Y values')
    axs[1, 1].set_ylabel('Frequency')
    axs[1, 1].set_yscale('log')
    
    plt.tight_layout()
    plt.show()

# Example usage
# pred: predictions numpy array of shape (batch_size, 9, 60)
# target: targets numpy array of shape (batch_size, 9, 60)
# Adjust bins as needed
stick_histograms(pred, target, 101, 101)


In [ ]:
def confusion_heatmap_none(predictions, targets, bins, label):
    # Clip and flatten the predictions and targets
    predictions = np.clip(predictions, -1, 1).flatten()
    targets = np.clip(targets, -1, 1).flatten()

    # Create a 2D histogram of the data
    heatmap, xedges, yedges = np.histogram2d(predictions, targets, bins=bins, range=[[-1, 1], [-1, 1]])
    
    # Plot the heatmap
    plt.figure(figsize=(10, 8))  # Increase figure size
    plt.imshow(heatmap, origin='lower', extent=[-1, 1, -1, 1], aspect='auto', cmap='viridis')
    
    # Adding color bar to understand the scale of counts
    plt.colorbar(label='Count')
    
    # Labels for axes
    plt.xlabel('Predictions')
    plt.ylabel('Targets')
    
    # Set x and y ticks to represent the edges of bins
    # plt.xticks((xedges[:-1] + xedges[1:])/2, labels=[f"{edge:.2f}" for edge in xedges[:-1]])
    # plt.yticks((yedges[:-1] + yedges[1:])/2, labels=[f"{edge:.2f}" for edge in yedges[:-1]])
    
    plt.title(f'Heatmap of {label} Predictions vs Targets')
    plt.show()

def confusion_heatmap_normalize_pred(predictions, targets, bins, label):
    predictions = np.clip(predictions   , -1, 1).flatten()
    targets = np.clip(targets   , -1, 1).flatten()

    # Calculate weights for normalization by predicted values
    # Count the occurrences of each predicted bin
    predicted_bins = np.digitize(predictions, bins=np.linspace(-1, 1, bins+1)) - 1
    predicted_bin_counts = np.bincount(predicted_bins, minlength=bins)
    weights_pred = 1 / predicted_bin_counts[predicted_bins]  # Weights are reciprocal of the counts

    # Create a 2D histogram of the data with weights
    heatmap, xedges, yedges = np.histogram2d(predictions, targets, bins=bins, range=[[-1, 1], [-1, 1]], weights=weights_pred)

    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    plt.imshow(heatmap, origin='lower', extent=[-1, 1, -1, 1], aspect='auto', cmap='viridis')
    plt.colorbar(label='Proportion')
    plt.xlabel('Predictions')
    plt.ylabel('Targets')
    plt.title(f'Heatmap {label} Normalized by Predicted')
    plt.show()

def confusion_heatmap_normalize_true(predictions, targets, bins, label):
    predictions = np.clip(predictions, -1, 1).flatten()
    targets = np.clip(targets, -1, 1).flatten()

    # Calculate weights for normalization by true values
    # Count the occurrences of each true bin
    true_bins = np.digitize(targets, bins=np.linspace(-1, 1, bins+1)) - 1
    true_bin_counts = np.bincount(true_bins, minlength=bins)
    weights_true = 1 / true_bin_counts[true_bins]  # Weights are reciprocal of the counts

    # Create a 2D histogram of the data with weights
    heatmap, xedges, yedges = np.histogram2d(predictions, targets, bins=bins, range=[[-1, 1], [-1, 1]], weights=weights_true)

    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    plt.imshow(heatmap, origin='lower', extent=[-1, 1, -1, 1], aspect='auto', cmap='viridis')
    plt.colorbar(label='Proportion')
    plt.xlabel('Predictions')
    plt.ylabel('Targets')
    plt.title(f'Heatmap {label} Normalized by True')
    plt.show()

def confusion_heatmaps(predictions, targets, bins, label):
    confusion_heatmap_none(pred[:, 0, :], target[:, 0, :], bins, label)
    confusion_heatmap_normalize_pred(pred[:, 0, :], target[:, 0, :],  bins, label)
    confusion_heatmap_normalize_true(pred[:, 0, :], target[:, 0, :],  bins, label)

bins = 30
# confusion_heatmaps(pred[:, 0, :], target[:, 0, :],  bins, 'JSTICK_X')
# confusion_heatmaps(pred[:, 1, :], target[:, 1, :],  bins, 'JSTICK_Y')

# confusion_heatmaps(np.arctan2(pred[:, 1, :], pred[:, 0, :]), np.arctan2(target[:, 1, :], target[:, 0, :]),  bins, 'J-Stick Angle')
# confusion_heatmaps(pred[:, 1, :]**2 + pred[:, 0, :]**2, target[:, 1, :]**2 + target[:, 0, :]**2,  bins, 'J-Stick Radius')

# confusion_heatmaps(pred[:, 2, :], target[:, 2, :],  bins, 'CSTICK_X')
# confusion_heatmaps(pred[:, 3, :], target[:, 3, :],  bins, 'CSTICK_Y')

# confusion_heatmaps(np.arctan2(pred[:, 3, :],  pred[:, 2, :]), np.arctan2(target[:, 3, :] , target[:, 2, :]),  bins, 'J-Stick Angle')
# confusion_heatmaps(pred[:, 2, :]**2 + pred[:, 3, :]**2, target[:, 2, :]**2 + target[:, 3, :]**2,  bins, 'J-Stick Radius')


Get a sense of how often the model predicts zeros. We look at the distribution of the number of frames a button is held over all games for the predictions and the labels.

In [ ]:
# convert the target and predicted to binary numpy
predicted_buttons = pred[:,-5:,:] >= .5
target_buttons = target[:,-5:,:] == 1
predicted_and_target_one = np.multiply(predicted_buttons ,target_buttons)

# sum over the buttons to get the number of frames the button is held
target_button_sum = np.sum(target_buttons, axis = 2)
pred_button_sum = np.sum(predicted_buttons, axis = 2)

prediction_and_target_one_sum = np.sum(predicted_and_target_one, axis = 2)
# take the difference between the number of frames the button is held in the prediction and the target
difference = target_button_sum - pred_button_sum

order = ['Trigger', 'Z', 'A', 'B', 'X_or_Y']
for i, label in enumerate(order):
    pred_unique, pred_counts = np.unique(pred_button_sum[:,i], return_counts=True)
    target_unique, target_counts = np.unique(target_button_sum[:,i], return_counts=True)
    diff_unique, diff_counts = np.unique(difference[:,i], return_counts=True)
    both_one_unique, both_one_counts = np.unique(prediction_and_target_one_sum[:,i], return_counts=True)

    plt.figure(figsize=(8, 4))
    plt.title(f'Number of ones in target {label}')
    plt.bar(target_unique[1:],target_counts[1:])
    # plt.bar(target_unique,target_counts)
    plt.show()
    
    plt.figure(figsize=(8, 4))
    plt.title(f'Number of ones predicted {label}')
    plt.bar(pred_unique[1:],pred_counts[1:])
    # plt.bar(pred_unique,pred_counts)
    plt.show()
    
    plt.figure(figsize=(8, 4))
    plt.title(f'Difference in number of ones in target and predicted {label}')
    plt.bar(diff_unique,diff_counts)
    plt.show()
    
    




In [ ]:
# convert the target and predicted to binary numpy
predicted_buttons = pred[:,-5:,:] >= .5
target_buttons = target[:,-5:,:] == 1
predicted_and_target_one = np.multiply(predicted_buttons ,target_buttons)

# sum over the buttons to get the number of frames the button is held
target_button_sum = np.sum(target_buttons, axis = 2)
pred_button_sum = np.sum(predicted_buttons, axis = 2)

prediction_and_target_one_sum = np.sum(predicted_and_target_one, axis = 2)
# take the difference between the number of frames the button is held in the prediction and the target
difference = target_button_sum - pred_button_sum

order = ['Trigger', 'Z', 'A', 'B', 'X_or_Y']
for i, label in enumerate(order):
    target_non_zero = target_button_sum[:,i] > 0
    # print(target_non_zero.shape)
    pred_unique, pred_counts = np.unique(pred_button_sum[target_non_zero,i], return_counts=True)
    target_unique, target_counts = np.unique(target_button_sum[target_non_zero,i], return_counts=True)
    diff_unique, diff_counts = np.unique(difference[target_non_zero,i], return_counts=True)
    both_one_unique, both_one_counts = np.unique(prediction_and_target_one_sum[target_non_zero,i], return_counts=True)

    plt.figure(figsize=(10, 4))
    plt.title(f'Number of ones in target {label} given target had a one')
    plt.bar(target_unique[1:],target_counts[1:])
    # plt.bar(target_unique,target_counts)
    # plt.show()
    
    # plt.figure(figsize=(10, 4))
    plt.title(f'Number of ones predicted {label} given target had a one')
    plt.bar(pred_unique[1:],pred_counts[1:])
    # plt.bar(pred_unique,pred_counts)
    plt.show()
    
    plt.figure(figsize=(10, 4))
    plt.title(f'Difference in number of ones in target and predicted {label} given target had a one')
    plt.bar(diff_unique,diff_counts)
    plt.show()
    


Once we get the distributions of the ones better (using a better loss function) and deal with the gradient problem we can focus in on the details of button presses. A confusion matrix where we look at the number of distinct times the button is pressed during a segment (it is pressed once but the prediction is two distinct presses). Given a button is pressed once and the prediction is that the button was pressed once, does the autoencoder predict the button was pressed at the same time? How many frames before the actual button was press does the autoencoder predict it was first pressed. How many frames is the end of the predicted button press off from the actual end of the button press.

Given any button was pressed, what is the probability that it predicted a button was pressed (not necessarily the one that was actually pressed). 
